# Add Dependences

In [10]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [12]:
parameters = Model(
    Cubic(
        μ₁       = 0.1,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.0000001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "BBB.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, [0.0 0.0 0.0]), Simulation(200, "BBB.xyz"))

In [13]:
println("Loading Data")
CUDA.@time X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
println("\n Generating Struct")
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)

## Adding Sum of Aggregates
println("\n Adding Aggregates from Position")
CUDA.@time SumAgg(Agg::Aggregate, parameters::Model)

# Simulating
println("\n Simulation")
simulate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

Loading Data
  0.008681 seconds (8.06 k CPU allocations: 317.703 KiB) (1 GPU allocation: 6.305 KiB, 0.62% memmgmt time)

 Generating Struct
  0.508550 seconds (367.91 k CPU allocations: 21.377 MiB, 7.66% gc time) (12 GPU allocations: 4.813 MiB, 29.33% memmgmt time)

 Loading Force Function
  0.000029 seconds (17 CPU allocations: 1.235 KiB)
A
 dding Aggregates from Position
  0.013986 seconds (6.82 k CPU allocations: 1.236 MiB) (20 GPU allocations: 4.850 MiB, 3.47% memmgmt time)

 Simulation


Progress:   3%|▊                        |  ETA: 0:17:11:40error in running finalizer: InterruptException()
maybe_collect at /buildworker/worker/package_linux64/build/src/julia_threads.h:313 [inlined]
jl_gc_pool_alloc at /buildworker/worker/package_linux64/build/src/gc.c:1204
default_stream at /home/nicomosty/.julia/packages/CUDA/DfvRa/lib/cudadrv/stream.jl:32 [inlined]
unsafe_finalize! at /home/nicomosty/.julia/packages/CUDA/DfvRa/src/array.jl:99
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2237 [inlined]
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2419
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1703 [inlined]
run_finalizer at /buildworker/worker/package_linux64/build/src/gc.c:278
jl_gc_run_finalizers_in_list at /buildworker/worker/package_linux64/build/src/gc.c:365
run_finalizers at /buildworker/worker/package_linux64/build/src/gc.c:394
jl_gc_collect at /buildworker/worker/package_linux64/build/src/gc.c:3260
gc at .